## Hand Made Neural Network

https://144.34.140.210:8579/notebooks/Git_Qiliang/ML_ANN_Gradiant_Decent/Backpropagation.ipynb

In [206]:
import pdb
bp = pdb.set_trace

In [2]:
import numpy as np

In [11]:
a = [[1,2,3,4,5,6,7,8,9,10],]
b = [12,3,4,5,1,8,5,12,98,2]
a = np.array(a)
b = np.array(b)

a*b

array([[ 12,   6,  12,  20,   5,  48,  35,  96, 882,  20]])

In [12]:
a*b.T

array([[ 12,   6,  12,  20,   5,  48,  35,  96, 882,  20]])

In [14]:
a.T*b

array([[ 12,   3,   4,   5,   1,   8,   5,  12,  98,   2],
       [ 24,   6,   8,  10,   2,  16,  10,  24, 196,   4],
       [ 36,   9,  12,  15,   3,  24,  15,  36, 294,   6],
       [ 48,  12,  16,  20,   4,  32,  20,  48, 392,   8],
       [ 60,  15,  20,  25,   5,  40,  25,  60, 490,  10],
       [ 72,  18,  24,  30,   6,  48,  30,  72, 588,  12],
       [ 84,  21,  28,  35,   7,  56,  35,  84, 686,  14],
       [ 96,  24,  32,  40,   8,  64,  40,  96, 784,  16],
       [108,  27,  36,  45,   9,  72,  45, 108, 882,  18],
       [120,  30,  40,  50,  10,  80,  50, 120, 980,  20]])

In [ ]:
import random
import uuid
import numpy as np



In [307]:
def relu(x):
    if x>0:
        return x
    else:
        return 0
activation_func = np.vectorize(relu)  
         
class Weight:
    def __init__(self,m,n):
        self.m = m
        self.n = n
        self.val = np.random.randn(m,n)
        self.normalize()
        
    def normalize(self):
        self.val = self.val/self.val.sum()

class Layer:
    def __init__(self,n):
        self.val = None
        self.input_weight = None
        self.output_weight = None
        self.pre_layer = None
        self.next_layer = None
        self.n = n
        self.delta_weight = None
        
        
    def full_connect(self,other):
        m = self.n
        n = other.n
        weight = Weight(m,n)
        self.output_weight = weight
        other.input_weight = weight
        
        self.next_layer = other
        other.pre_layer = self
    
    def stats(self):
        print("n:",self.n)
        if self.output_weight:
            print('Weight:',self.output_weight.val.shape)
        print()
            
    def pull_in(self,input_list):
        self.val = np.array([input_list,])
        self.val = self.val/self.val.sum()
            
    def forwarding(self):
        val = self.val.dot(self.output_weight.val)
        try:
            self.next_layer.val = activation_func(val)      
        except:
#             bp()
            raise
        
    def backwarding(self, error_term, learning_rate):
        
        pre = self.pre_layer
        K1 = pre.output_weight.val
#         print(K1.shape)
        while(pre.pre_layer):
            pre = pre.pre_layer
            K1 = pre.output_weight.val.dot(K1)
            
        K1 = pre.val.dot(K1)
#         print('K1:',K1.shape)


        next_layer = self.next_layer
        if next_layer.output_weight:
            K2 = next_layer.output_weight.val
            while(next_layer.next_layer.output_weight):
                next_layer = next_layer.next_layer
                K2 = K2.dot(next_layer.output_weight.val) 
#             print("K2:",K2.shape)
        else:
            K2 = np.array([[1]])
            
            
#         return K1,K2
#         bp()
        derivative = K1.T.dot(K2.T)
#         print("Derivative:",derivative.shape)
#         return derivative
    
        delta_weight = learning_rate*(-1)*(error_term)*derivative
#         bp()
        if self.delta_weight:
            self.delta_weight += delta_weight
        else:
            self.delta_weight = delta_weight
    
    def update_weights(self):
        self.output_weight.val += self.delta_weight
        self.delta_weight = None
    
class Network:
    def __init__(self,input_layer,output_layer):
        self.input_layer = input_layer
        self.output_layer = output_layer
              
    def batch_forwarding(self):
        cur = self.input_layer
        while(cur.next_layer):
            cur.forwarding()
            cur = cur.next_layer
             
    def get_error_term(self,ground_truth_list):      
        error = self.output_layer.val-ground_truth_list
        return error
    
    def batch_backwarding(self, error_term, learning_rate=0.01):
        cur = self.input_layer.next_layer
        while(cur and cur.output_weight):
            cur.backwarding(error_term, learning_rate)
            cur.update_weights()
            cur = cur.next_layer
            
    def one_train(self,x,y):
        input_layer.pull_in(x)
        self.batch_forwarding()
        error_term = self.get_error_term(y)
        self.batch_backwarding(error_term,0.001)
        

    def batch_train(self,X,Y,batch_size = 20):
        for x,y in zip(X,Y):
            self.one_train(x,y)
        
    def predict(self,x):
        input_layer.pull_in(x)
        self.batch_forwarding()
        return self.output_layer.val
        
# Networks
input_layer = Layer(10)
l1 = Layer(10)
l2 = Layer(50)
l3 = Layer(50)
l4 = Layer(10)
output_layer = Layer(1)


input_layer.full_connect(l1)
l1.full_connect(l2)
l2.full_connect(l3)
l3.full_connect(l4)
l4.full_connect(output_layer)

# x = random.choices(range(100), k=10)

# print(x)
# y = np.array([[1]])

# nk = Network(input_layer,output_layer)
# nk.one_train(x,y)


In [308]:
# nk = Network(input_layer,output_layer)

In [309]:
# x = np.array(random.choices(range(100), k=10))
# x = x/x.sum()
# vector = [2, 24, 31, 81, 2, 76, 57, 81, 79, 76]
# vector = np.array(vector)
# vector = vector/vector.sum()
# # y = x.dot(vector) + random.randrange(-10,10)
# y = x.dot(vector) + (random.random()-0.5)*0.2

# print(y)

In [310]:
vector = [2, 24, 31, 81, 2, 76, 57, 81, 79, 76]
vector = np.array(vector)
vector = vector/vector.sum()

X = []
Y = []
for i in range(100):
    x = np.array(random.choices(range(100), k=10))
    x = x/x.sum()
    y = x.dot(vector) + (random.random()-0.5)*0.2
    X.append(x)
    Y.append(y)

In [311]:
nk = Network(input_layer,output_layer)
nk.batch_train(X,Y)

In [324]:
l4.output_weight.val

array([[-0.26996883],
       [ 0.66489401],
       [ 0.41041368],
       [-0.0636539 ],
       [ 1.3391445 ],
       [ 0.17079538],
       [-0.61470007],
       [ 0.34895184],
       [-0.39202483],
       [-0.59448242]])

In [323]:
l3.output_weight.val

array([[-3.81257986e-02, -3.09209161e-03, -2.26457649e-02,
         6.83607079e-03, -8.56105820e-03, -5.00895975e-02,
        -2.98922840e-02,  1.82742320e-02,  5.49571484e-02,
         2.68968742e-02],
       [-3.81986579e-02,  2.72557544e-02, -6.88306600e-05,
        -1.45715132e-02,  2.10002406e-02,  8.70399832e-03,
        -3.69505519e-02, -2.10710609e-02, -2.82383049e-02,
         2.16430746e-02],
       [-1.27106887e-02,  7.04981246e-02, -1.24826796e-03,
        -4.19335962e-02, -1.39949304e-02,  7.25904447e-02,
        -3.69564184e-02,  3.17880057e-02, -2.12089850e-02,
        -1.82891866e-02],
       [ 4.33577777e-02,  4.85483422e-02, -5.82453451e-02,
        -6.66502171e-03, -4.54504002e-02,  6.43248873e-02,
         2.10960591e-02,  2.14010550e-02,  3.31012056e-02,
        -4.68095741e-02],
       [-1.19561336e-02,  5.45343308e-03,  3.74046923e-02,
         1.81935309e-02,  2.75425270e-02, -1.85663826e-02,
         2.58637234e-02, -3.67274894e-02, -1.22354020e-02,
        -2.

In [322]:
l2.output_weight.val

array([[-0.01458435, -0.03735993,  0.01323616, ..., -0.06182775,
        -0.00039428,  0.02236964],
       [ 0.03157045,  0.02544207,  0.00759125, ..., -0.03845227,
        -0.01945115, -0.00843746],
       [-0.00669911, -0.0178306 , -0.01973558, ..., -0.01083613,
         0.02843392,  0.02845818],
       ...,
       [ 0.00242258,  0.00805969,  0.04567194, ...,  0.03203191,
         0.00524877,  0.01476885],
       [ 0.00048926, -0.0110352 ,  0.00574007, ...,  0.02051919,
        -0.00583478,  0.01146048],
       [ 0.03245254, -0.00108037,  0.00041602, ..., -0.0022537 ,
         0.01840172, -0.00441561]])

In [321]:
l1.output_weight.val

array([[ 2.19312574, -0.64601977,  0.15641916,  1.26808349, -0.49851728,
        -0.90221019,  1.37464117,  0.36429519,  0.25781248, -0.20139758,
        -1.37638773,  0.02258913, -0.05136871, -1.29786979,  0.56411755,
        -1.04784905,  0.41917256, -0.06845508, -1.06535451, -0.84201641,
        -1.39427216, -1.07881703,  1.20461365,  1.19989188, -2.51868383,
         0.58817437,  0.27033754,  1.98260174,  0.19750839,  2.06840073,
         0.75169812, -0.94790495, -0.37093517,  0.08564069, -1.33733549,
         0.08244358, -0.82333015, -0.23462689, -0.03605369, -0.06288988,
        -0.01511137, -0.78963987,  1.13260949, -0.15632028,  0.96178992,
        -0.50625476, -0.28070834, -0.00506774, -0.15941545,  0.63853252],
       [-1.37566316, -0.67284093,  0.94931018, -0.56005391,  0.09878266,
         0.27199278, -0.83376773, -0.11965969, -0.8180054 , -1.80203455,
         1.240273  ,  2.48360163,  0.06965277,  2.09252576,  1.35715635,
         1.02623457,  0.65682879, -1.08926929,  0.

In [320]:
x = np.array(random.choices(range(100), k=10))
x = x/x.sum()
y_real = x.dot(vector) + (random.random()-0.5)*0.2
y_predict = nk.predict(x)

print('x:',x)
print('y_real:',y_real)
print('y_predict:',y_predict)

x: [0.07225434 0.06069364 0.00867052 0.28612717 0.06647399 0.03179191
 0.08959538 0.24566474 0.06936416 0.06936416]
y_real: 0.0826030171934313
y_predict: [[0]]


In [223]:
nk.predict(x)

array([[0]])

In [214]:
l1.stats()
l2.stats()
l3.stats()
l4.stats()

n: 10
Weight: (10, 50)

n: 50
Weight: (50, 50)

n: 50
Weight: (50, 10)

n: 10
Weight: (10, 1)



In [215]:
print('input_layer:',[round(x,2) for x in input_layer.val[0]])
print('l1:',[round(x,2) for x in l1.val[0]])
print('l2:',[round(x,2) for x in l2.val[0]])
print('l3:',[round(x,2) for x in l3.val[0]])
print('l4:',[round(x,2) for x in l4.val[0]])
print('output_layer:',[round(x,2) for x in output_layer.val[0]])

input_layer: [85, 78, 24, 50, 51, 95, 1, 22, 51, 65]
l1: [24.03, 7.7, 37.9, 0.0, 45.19, 0.0, 82.94, 0.0, 0.0, 0.0]
l2: [2.99, 0.0, 3.95, 0.0, 0.0, 0.0, 0.0, 0.0, 1.7, 0.0, 0.0, 6.15, 0.0, 12.57, 25.25, 15.12, 0.0, 0.0, 0.0, 11.79, 19.67, 28.17, 0.0, 5.11, 6.01, 0.0, 0.0, 2.56, 13.08, 35.02, 0.0, 3.71, 62.84, 28.64, 51.16, 0.54, 0.0, 33.64, 0.0, 19.34, 0.0, 0.0, 23.39, 15.72, 0.0, 0.52, 15.9, 0.0, 0.0, 20.21]
l3: [0, 0, 2, 0, 0, 1, 0, 0, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 0, 0, 2]
l4: [0.57, 0.0, 0.0, 0.38, 0.0, 0.0, 0.0, 0.0, 0.3, 0.08]
output_layer: [0.15]


In [216]:
l1.delta_weight

array([[ 3.03586222e-04,  1.34174545e-04,  1.22835635e-03,
         7.51947139e-04,  2.10621331e-04, -2.25281513e-04,
         4.58187184e-04, -2.46946156e-04,  5.46989560e-04,
        -3.43687642e-04, -1.59704308e-03,  1.14776961e-04,
        -2.20332758e-04,  1.64067880e-03, -6.79567001e-05,
        -5.21673877e-04,  8.03791896e-04,  1.42830370e-03,
        -1.02715743e-04,  1.80930581e-04, -1.24633486e-03,
        -8.01996503e-04, -1.80922030e-04, -3.42926879e-04,
         1.10753069e-04,  7.04940749e-04,  9.13847454e-05,
        -9.89001596e-04,  3.15081050e-04,  7.34962703e-04,
        -1.31275881e-03,  1.44556152e-03, -1.49704793e-04,
         1.29837762e-03,  1.40979862e-03, -5.61983500e-04,
         9.43563031e-04, -6.12721008e-05,  6.11976414e-04,
        -7.71239116e-04,  7.29477931e-05,  9.56180302e-04,
        -4.16107126e-04,  9.52857139e-04,  5.78770502e-04,
         7.90586184e-04,  1.39235811e-03,  6.02323722e-04,
        -3.59216151e-04, -1.20803097e-04],
       [ 9.72

In [217]:
l2.delta_weight

array([[ 1.81545014e-03, -5.00561458e-03,  6.79693171e-03, ...,
        -5.55600742e-03, -4.51932771e-04,  1.76388167e-03],
       [-6.45120604e-03,  1.77874624e-02, -2.41529118e-02, ...,
         1.97432846e-02,  1.60594409e-03, -6.26795740e-03],
       [ 3.19865500e-03, -8.81942930e-03,  1.19755642e-02, ...,
        -9.78917051e-03, -7.96263687e-04,  3.10779615e-03],
       ...,
       [ 1.17568665e-04, -3.24163916e-04,  4.40169727e-04, ...,
        -3.59807391e-04, -2.92671948e-05,  1.14229089e-04],
       [-5.76694087e-03,  1.59007856e-02, -2.15910658e-02, ...,
         1.76491580e-02,  1.43560515e-03, -5.60312901e-03],
       [ 7.58638396e-03, -2.09174096e-02,  2.84029469e-02, ...,
        -2.32173855e-02, -1.88853192e-03,  7.37089022e-03]])

In [159]:
l1.stats()

n: 10


In [160]:
# l2.output_edges
l2.stats()

n: 50


In [37]:
l3.stats()

Num nodes: 50
Input Edges: 2500
Output Edges: 50


In [38]:
l4.stats()

Num nodes: 1
Input Edges: 50
Output Edges: 0
